In [4]:
import requests
import pandas as pd
from urllib.parse import quote
from bs4 import BeautifulSoup
import re

In [5]:
### 한글 숫자 필터링 함수
def test(s):
  hangul = re.compile('[^ ㄱ-ㅣ가-힣0-9+]') # 한글과 띄어쓰기를 제외한 모든 글자 # hangul = re.compile('[^ \u3131-\u3163\uac00-\ud7a3]+') # 위와 동일 
  result = hangul.sub('', s) # 한글과 띄어쓰기를 제외한 모든 부분을 제거 
  return (result)


In [63]:
name = '명량'
url = f'https://movie.naver.com/movie/search/result.naver?query={name}&section=all'      # point 사이트 내에서 평점 , point 부분이 바뀌면 그 밖에 리뷰, 주요정보 등 다른 정보가 화면내 표시
req = requests.get(url)
html = req.text
#html

In [64]:
soup =BeautifulSoup(html, 'html.parser')

In [65]:
inf_m = soup.select_one("#netizen_point_graph > div > strong")
inf = inf_m.text       ### 네이버 영화측에서 분석한 성별 연령층 정보 + 영화의 특징
inf

AttributeError: ignored

In [95]:
star = soup.select_one("div.main_score div.score a div.star_score span.st_off span.st_on")
star.text       ### 관람객 평점

'관람객 평점 없음'

In [96]:
date = soup.select_one('#content > div.article > div.mv_info_area > div.mv_info > dl > dd > p')
date = date.text
m = date.split('\n\n') ## \n\n으로 구분 돼 장르, 나라, 시간, 개봉일이 표시되는 것 같습니다.
jangru = test(m[1])
contry = test(m[2])
time = test(m[3])
date = test(m[4])
print(jangru,contry,time,date)

멜로로맨스 코미디 가족 미국 103분   19950513 개봉


In [1]:
name = '명량'
url = f'https://movie.naver.com/movie/search/result.naver?query={name}&section=all'

In [88]:
##### 영화 제목
name = '명량' #### 영화제목입력
url = f'https://movie.naver.com/movie/search/result.naver?query={name}&section=all'
req = requests.get(url)
html = req.text
soup =BeautifulSoup(html, 'html.parser') ## 영화제목 검색된 창
title_list =[]
star_list = []
jangru_list = []
country_list = []
time_list =[]
date_list = []
info_list = []
for i in range(1,5):
  site_num = soup.select_one(f'#old_content > ul.search_list_1 > li:nth-of-type({i}) > dl > dt > a')      ### 중복되는 영화들 중 순서대로 4개만 크롤링
  ######### 영화 정보 사이트로 이동
  if site_num != None:
    new_name = site_num['href']     # 영화 사이트 주소 가져오기
    new_url = f'https://movie.naver.com/{new_name}'
    new_url = new_url.replace('basic','point')      # 사이트에서 평점을 클릭 해야함
    new_req = requests.get(new_url)
    new_html = new_req.text
    s =BeautifulSoup(new_html, 'html.parser')
    ##### 정보 가져오기 ########
    title = s.select_one('#content > div.article > div.mv_info_area > div.mv_info > h3 > a')
    title = title.text     ### 영화 제목
    inf_m = s.select_one("#netizen_point_graph > div > strong")
    inf = inf_m.text       ### 네이버 영화측에서 분석한 성별 연령층 정보 + 영화의 특징
    star = s.select_one("div.main_score div.score a div.star_score span.st_off span.st_on")
    star = star.text       ### 관람객 평점
    date = s.select_one('#content > div.article > div.mv_info_area > div.mv_info > dl > dd > p')
    date = date.text
    m = date.split('\n\n') ## \n\n으로 구분 돼 장르, 나라, 시간, 개봉일이 표시되는 것 같습니다.
    jangru = test(m[1])
    country = test(m[2])
    time = test(m[3])
    date = test(m[4])
    title_list.append(title)
    star_list.append(star)
    jangru_list.append(jangru)
    country_list.append(country)
    time_list.append(time)
    date_list.append(date)
    info_list.append(inf)
  else:
    pass

title_list

['명량: 회오리 바다를 향하여', '명량']

In [84]:
##### 영화 제목
name = '명량' #### 영화제목입력
url = f'https://movie.naver.com/movie/search/result.naver?query={name}&section=all'
req = requests.get(url)
html = req.text
soup =BeautifulSoup(html, 'html.parser') ## 영화제목 검색된 창
title_list =[]
star_list = []
jangru_list = []
country_list = []
time_list =[]
date_list = []
info_list = []
i= 3
site_num = soup.select_one(f'#old_content > ul.search_list_1 > li:nth-of-type({i}) > dl > dt > a')      ### 중복되는 영화들 중 순서대로 4개만 크롤링
  ######### 영화 정보 사이트로 이동
site_num != None

False

In [60]:
title = soup.select_one('#content > div.article > div.mv_info_area > div.mv_info > h3 > a')
title = title.text

'명량: 회오리 바다를 향하여'

In [72]:
i =1
site_num = soup.select(f'#old_content > ul.search_list_1 > li:nth-of-type({i}) > dl > dt > a')
site_num != []

True